In [1]:
import json
import pandas as pd
import sys
import os

from mapboxgl.viz import *
from mapboxgl.utils import *
from sodapy import Socrata

## Set Mapbox token
Set a MAPBOX_ACCESS_TOKEN environment variable or copy/paste your token  
If you do not have a Mapbox access token, sign up for an account at https://www.mapbox.com/  
If you already have an account, you can grab your token at https://www.mapbox.com/account/  

In [2]:
# set Mapbox access token
token = os.getenv('MAPBOX_ACCESS_TOKEN')

## Get data
We're grabbing [real-time fire 911 calls in Seattle, WA](https://dev.socrata.com/foundry/data.seattle.gov/grwu-wqtk) using the `sodapy` python client for the Socrata Open Data API.  
Search for more open data [here](https://opendata.socrata.com/).

In [3]:
# configure request
domain = 'data.seattle.gov'
resource = 'grwu-wqtk'
client = Socrata(domain, None)

In [4]:
# get count of records by incident type
json_data = json.dumps(client.get(resource, content_type='json'))
df_counts = pd.read_json(json_data).groupby('type')['type'].count()
# view top 5 incident types
df_counts.sort_values(ascending=False).head(5)

type
Aid Response           576
Medic Response         110
Trans to AMR            67
Auto Fire Alarm         37
Aid Response Yellow     30
Name: type, dtype: int64

In [5]:
# create a dataframe of the top 5 incident types
json_data = json.dumps(client.get('grwu-wqtk', 
                                  content_type='json', 
                                  where="type='Aid Response' "\
                                        "or type='Medic Response' " \
                                        "or type='Trans to AMR' " \
                                        "or type='Auto Fire Alarm' " \
                                        "or type='Aid Response Yellow '"
                                 )
                      )
df = pd.read_json(json_data)

## Prepare data

In [6]:
# Clean up by dropping null rows
df.dropna(axis=1, how='all', inplace=True)
df.head(2)

,address,datetime,incident_number,latitude,longitude,report_location,type
0,3rd Av / James St,2017-11-07 05:07:00,F170115735,47.602769,-122.331409,"{u'type': u'Point', u'coordinates': [-122.3314...",Aid Response
1,15th Av / E Olive St,2017-11-07 05:01:00,F170115731,47.616451,-122.312761,"{u'type': u'Point', u'coordinates': [-122.3127...",Medic Response


## Visualize

In [7]:
# select properties
latitude_col = 'latitude'
longitude_col = 'longitude'
properties = ['address', 'datetime', 'incident_number', 'type']

In [8]:
# Create geojson data object
geo_json = df_to_geojson(df.fillna(''), 
                         lat=latitude_col, 
                         lon=longitude_col,
                         properties=properties,
                         precision=4
                        ) 

In [9]:
# create visualization
# Assign color stops
category_color_stops = [
    ['Aid Response', 'rgb(211,47,47)'],  
    ['Medic Response', 'rgb(81,45,168)'], 
    ['Trans to AMR', 'rgb(2,136,209)'], 
    ['Auto Fire Alarm', 'rgb(139,195,74)'], 
    ['Aid Response Yellow', 'rgb(255,160,0)'], 
]

viz = CircleViz(geo_json,
                opacity=0.8,
                zoom=10,
                center= [-122.3, 47.6],
                access_token=token)

# Marker color-related attributes
viz.color_property = 'type'
viz.default_color = 'grey'
viz.color_function_type = 'match'
viz.color_stops = category_color_stops

viz.show()